In [21]:
import numpy as np
import pandas as pd

Import new dataset

In [22]:
users = pd.read_csv(r"D:\dataset\encoding\user_frame.csv")
anime = pd.read_csv(r"D:\dataset\encoding\anime_frame.csv")
ratings = pd.read_csv(r"D:\dataset\encoding\ratings_frame.csv")

In [23]:
users

# encoding users: 
# 1. encode fav_genres as bag of words: use term frequency vector for each
# 2. encode other features
# 3. join encoded genres with the rest of the vectorised features
# need to translate genres that user is asked to anime genres


,user_id,username,user_completed,experience,gender,birth_date,stats_episodes,generation,fav_genres,fav_anime_period
0,2255153,karthiga,49,Active,Female,1990,3391,Millenials,"['Comedy', 'Shounen', 'Action']",Classic
1,37326,Damonashu,195,Active,Male,1991,4903,Millenials,"['Drama', 'Action', 'Sci-Fi']",New-Gen
2,228342,bskai,414,Veteran,Male,1990,9701,Millenials,"['Comedy', 'Romance', 'School']",Classic
3,61677,Slimak,224,Veteran,Male,1988,7447,Millenials,"['Action', 'Super', 'Power']",Classic
4,2485327,MistButterfly,3923,Veteran,Female,1992,39703,Millenials,"['Action', 'Historical', 'Sci-Fi']",New-Gen
...,...,...,...,...,...,...,...,...,...,...
9297,363965,purgatorio,727,Veteran,Male,1993,10701,Millenials,"['Comedy', 'Ecchi', 'Action']",New-Gen
9298,5180847,Kagekori-chan,497,Veteran,Female,2001,11192,Gen Z,"['Action', 'Supernatural', 'Comedy']",New-Gen
9299,332815,Sicka,1912,Veteran,Male,1986,13168,Millenials,"['Comedy', 'Adventure', 'Action']",Classic
9300,25554,winry-chan,7,Regular,Female,1994,662,Millenials,"['Comedy', 'Shounen', 'Adventure']",Classic


In [24]:
anime

,anime_id,title,type,airing,scored_by,members,studio,genre,aired_from_year,time_period,fame
0,11013,Inu x Boku SS,TV,False,139250,283882,David Production,"['Comedy', ' Supernatural', ' Romance', ' Shou...",2012,New-Gen,famous
1,2104,Seto no Hanayome,TV,False,91206,204003,Gonzo,"['Comedy', ' Parody', ' Romance', ' School', '...",2007,Classic,semi-famous
2,5262,Shugo Chara!! Doki,TV,False,37129,70127,Satelight,"['Comedy', ' Magic', ' School', ' Shoujo']",2008,Classic,well-known
3,721,Princess Tutu,TV,False,36501,93312,Hal Film Maker,"['Comedy', ' Drama', ' Magic', ' Romance', ' F...",2002,Classic,semi-famous
4,12365,Bakuman. 3rd Season,TV,False,107767,182765,J.C.Staff,"['Comedy', ' Drama', ' Romance', ' Shounen']",2012,New-Gen,semi-famous
...,...,...,...,...,...,...,...,...,...,...,...
3502,34522,"Wake Up, Girls! Shin Shou",TV,False,1848,7772,Millepensee,"['Music', ' Drama']",2017,New-Gen,niche
3503,24415,Kuroko no Basket 3rd Season,TV,False,174065,287422,Production I.G,"['Comedy', ' Sports', ' School', ' Shounen']",2015,New-Gen,famous
3504,478,Sousei no Aquarion,TV,False,23090,55092,"Satelight, Production Reed","['Action', ' Mecha', ' Romance', ' Super Power...",2005,Classic,well-known
3505,4948,Shounen Sarutobi Sasuke,Movie,False,314,815,Toei Animation,"['Adventure', ' Fantasy']",1959,Classic,unknown


In [25]:
ratings

,username,anime_id,title,genre,type,my_score,my_watched_episodes,aired_from_year,time_period
0,--FallenAngel--,164,Mononoke Hime,"['Action', ' Adventure', ' Fantasy']",Movie,10,1,1997,Classic
1,--FallenAngel--,5341,Ookami to Koushinryou II,"['Adventure', ' Historical', ' Romance', ' Fan...",TV,10,12,2009,Classic
2,--FallenAngel--,877,Nana,"['Music', ' Slice of Life', ' Comedy', ' Drama...",TV,10,47,2006,Classic
3,--FallenAngel--,3731,Itazura na Kiss,"['Comedy', ' Romance', ' Shoujo']",TV,10,25,2008,Classic
4,--FallenAngel--,1914,Saiunkoku Monogatari 2nd Season,"['Adventure', ' Comedy', ' Drama', ' Fantasy',...",TV,10,39,2007,Classic
...,...,...,...,...,...,...,...,...,...
92927,zzs,4898,Kuroshitsuji,"['Action', ' Comedy', ' Demons', ' Fantasy', '...",TV,8,24,2008,Classic
92928,zzs,60,Chrno Crusade,"['Action', ' Historical', ' Demons', ' Superna...",TV,8,24,2003,Classic
92929,zzs,167,Scrapped Princess,"['Adventure', ' Comedy', ' Drama', ' Fantasy',...",TV,8,24,2003,Classic
92930,zzs,1974,Glass no Kamen (2005),"['Drama', ' Shoujo']",TV,8,51,2005,Classic
